In [1]:
import pydicom
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import nibabel as nib

In [2]:
def load_dicom_series(dicom_folder):
    """Load and sort DICOM image slices from a folder and detect the RTSS file."""
    dicom_files = []
    rtss_file = None
    
    for file in os.listdir(dicom_folder):
        path = os.path.join(dicom_folder, file)
        try:
            dicom = pydicom.dcmread(path)
            
            if dicom.Modality == "RTSTRUCT":
                rtss_file = dicom  # Store the RT Structure Set file
            elif hasattr(dicom, "ImagePositionPatient"):  # Only keep image slices
                dicom_files.append(dicom)
        except Exception as e:
            print(f"Skipping {file}: {e}")

    dicom_files.sort(key=lambda d: d.ImagePositionPatient[2])  # Sort by slice position
    return dicom_files, rtss_file

In [3]:
# Paths
dicom_folder = r"C:\Users\r.joshi\Downloads\01_11_2024\7139000005\COMBI\L"

In [ ]:
# Load dicom series and RTSS
dicom_series, rtss = load_dicom_series(dicom_folder)

Skipping original_dicom_volume.nii.gz: File is missing DICOM File Meta Information header or the 'DICM' prefix is missing from the header. Use force=True to force reading.


In [6]:
dicom_volume = []
for dicom in dicom_series:
    if dicom.Modality == 'US':
        dicom_volume.append(dicom.pixel_array)

In [35]:
dicom_volume = np.array(dicom_volume)
dicom_volume = np.transpose(dicom_volume, (2, 1, 0))

In [36]:
def get_slice_spacing(dicom_series):
    """Calculate slice spacing using ImagePositionPatient."""
    if len(dicom_series) > 1:
        z_positions = [float(d.ImagePositionPatient[2]) for d in dicom_series]
        z_positions.sort()  # Ensure slices are ordered
        slice_spacing = np.mean(np.diff(z_positions))  # Compute mean spacing
    else:
        slice_spacing = 1.0  # Default fallback if only one slice is present
    return slice_spacing

In [ ]:
reference_slice = dicom_series[0]
# Get Pixel Spacing
pixel_spacing = np.array(reference_slice.PixelSpacing, dtype=np.float32)  # [row_spacing, col_spacing]

# Get Slice Spacing
slice_spacing = get_slice_spacing(dicom_series)

pixel_spacing, slice_spacing

(array([0.116958, 0.124223], dtype=float32), np.float64(0.9852941176470471))

In [38]:
# Create Affine Matrix
affine = np.eye(4)
affine[0, 0] = pixel_spacing[0]  # X-axis spacing
affine[1, 1] = pixel_spacing[1]  # Y-axis spacing
affine[2, 2] = slice_spacing     # Z-axis spacing (computed from slice positions)

In [39]:
affine

array([[0.116958  , 0.        , 0.        , 0.        ],
       [0.        , 0.124223  , 0.        , 0.        ],
       [0.        , 0.        , 0.98529412, 0.        ],
       [0.        , 0.        , 0.        , 1.        ]])

In [40]:
def save_as_nifti(image_data, output_path, affine=np.eye(4)):
    """Save the image data as a NIfTI file."""
    # print(output_path, affine)
    img = nib.Nifti1Image(image_data, affine)
    nib.save(img, output_path)

In [43]:
# Define output file path
output_nifti_path = os.path.join(dicom_folder, "dicom_volume.nii.gz")

# Use your save_as_nifti function
save_as_nifti(dicom_volume, output_nifti_path, affine)

In [44]:
def get_contour_data(rtss, structure_name=None):
    """Extract contour data for a given structure from RTSS. If structure_name is None, get data for all structures."""
    contours_per_structure = {}
    
    for roi in rtss.StructureSetROISequence:
        structure_name = roi.ROIName  # Structure name
        roi_number = roi.ROINumber

        contours_per_slice = {}
        
        # Iterate through the ROIContourSequence to find contour data for the structure
        for roi_contour in rtss.ROIContourSequence:
            if roi_contour.ReferencedROINumber == roi_number:
                
                # Check if 'ContourSequence' exists and extract data if present
                if hasattr(roi_contour, 'ContourSequence'):
                    for contour in roi_contour.ContourSequence:
                        z_pos = round(contour.ContourData[2], 2)  # Z position
                        points = np.array(contour.ContourData).reshape(-1, 3)[:, :2]  # X, Y only
                        contours_per_slice.setdefault(z_pos, []).append(points)

        # If the structure has contours, add it to the results
        if contours_per_slice:
            contours_per_structure[structure_name] = contours_per_slice

    return contours_per_structure

In [ ]:
def create_binary_mask(dicom, contours, boundary_margin=5):
    """Convert contours into a binary mask matching DICOM image dimensions."""
    image_shape = dicom.pixel_array.shape
    mask = np.zeros(image_shape, dtype=np.uint8)

    if contours:
        pixel_spacing = dicom.PixelSpacing
        origin = dicom.ImagePositionPatient[:2]

        for contour in contours:
            pixel_points = []
            for point in contour:
                x = (point[0] - origin[0]) / pixel_spacing[0]
                y = (point[1] - origin[1]) / pixel_spacing[1]

                # Clip points to stay within image bounds
                x = np.clip(int(x), 0, image_shape[1] - 1)
                y = np.clip(int(y), 0, image_shape[0] - 1)

                # print(x, y)
                pixel_points.append((x, y))

            if pixel_points:  # Only proceed if there are valid points
                pixel_points = np.array(pixel_points, np.int32)
                pixel_points = pixel_points.reshape((-1, 1, 2))  # Format for OpenCV

                # Fill the polygon into the mask
                cv2.fillPoly(mask, [pixel_points], 255)

    return mask

In [81]:
contours_per_structure = get_contour_data(rtss)

In [85]:
for structure_name, contours_per_slice in contours_per_structure.items():
    
    mask_volume = []
    slice_shape = reference_slice.pixel_array.shape

    for dicom in dicom_series:
        if dicom.Modality == 'US':
            z_pos = round(dicom.ImagePositionPatient[2], 2)
            contours = contours_per_slice.get(z_pos, [])
            
            mask = create_binary_mask(dicom, contours)

            print(np.count_nonzero(mask))

            mask_volume.append(mask)

    mask_volume = np.array(mask_volume)
    mask_volume = np.transpose(mask_volume, (2, 1, 0))
    # Save the mask as a NIfTI file
    output_mask_path = os.path.join(dicom_folder, f"{structure_name}_mask.nii.gz")
    save_as_nifti(mask_volume, output_mask_path, affine)
    print(f"Saved mask for {structure_name} to {output_mask_path}")

0
0
0
0
0
0
0
0
0
0
0
0
0
0
300 309
296 315
292 322
288 329
285 337
282 344
278 351
273 358
267 365
261 371
256 378
253 385
251 393
253 401
256 408
260 415
265 422
271 428
277 433
284 438
292 440
301 441
310 441
319 440
329 438
338 438
346 437
354 437
362 436
371 435
379 435
387 434
395 433
404 434
413 434
421 435
429 435
437 432
444 428
451 422
456 416
461 409
464 402
465 394
465 387
464 380
463 372
461 365
458 358
456 351
452 344
449 337
445 330
441 324
437 317
432 311
427 305
420 300
414 295
407 290
400 286
392 282
385 278
378 276
370 274
362 273
354 274
346 276
339 279
332 283
325 287
318 292
312 297
306 303
26603
322 284
329 280
337 276
345 273
353 271
362 271
371 271
378 273
386 276
394 279
402 283
410 288
417 292
423 297
430 303
436 309
441 315
446 322
454 335
458 342
462 349
465 357
467 364
469 372
471 380
472 388
472 396
471 404
468 413
462 421
455 428
446 434
438 438
430 440
421 440
413 439
404 438
396 438
387 438
330 443
311 446
303 446
294 446
284 444
276 440
269 435
260 42